### 本文件用于选取中国211 985 以及美国CSRanking 在前160名的学校的校徽，作为优质数据进行模型和算法的筛选。其后还进行了数据的预处理，包括切除纯色的边框，保存为RGB格式以及Resize
### This notebook is used to select the Chinese School of 211 and 985 program as well as the USA School from CSRanking 160th above as high equality data to process model and algorithm selection and comparetion. There are also data preprocess including cut the pure color edge, save with .jpeg(rgb) format and resize

---------

## 中国选取211 985

In [2]:
import pandas as pd
import os
from shutil import copyfile
import re

In [11]:
df = pd.read_excel('../211 985.xlsx', names = ['names'])

In [4]:
df = pd.read_excel('../CSRanking_USA.xlsx')

In [6]:
os.chdir('./data_with_class/USA')
os.getcwd()

'/home/gs/git-repos/University-Logo-Deep-Learning-Community-Detection/data_with_class/USA'

In [12]:
os.getcwd()

'/home/gs/git-repos/University-Logo-Deep-Learning-Community-Detection/src'

In [ ]:
for sch in df.names:
    try:
        copyfile('../' + sch + '_China.jpeg', '../../Selected_data_raw/China/' + sch + '_China.jpeg')
    except:
        print(sch)

--------
> 由于原始数据并没有保存，所以通过直接复制的办法并不好，全部都是已经经过resize的数据    
> 所以针对美国的数据进行重新爬取
--------

# 重新爬取CSRanking 上USA前162名的学校的校徽

In [15]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import html5lib
from urllib.request import urlretrieve

In [14]:
keywords = '{school} seal'

In [1]:
def getUrl(browser, universityName, keywords):
    browser.get('https://www.google.com/imghp')
    google_input = browser.find_element_by_class_name('gLFyf')
    google_input.send_keys(keywords.format(school = universityName))
    google_input.send_keys(Keys.ENTER)
    first_result = browser.find_element_by_css_selector('#rg img')
    src = first_result.get_attribute('src')
    return src

def downloadImg(url, directory, name, datatype):
    from urllib.request import urlretrieve
    urlretrieve(url, directory + name + '.' + datatype)

def main(keywords, country, universityList):
    count = 0
    browser = webdriver.Chrome()
    for school in universityList:
        url = ''
        if count == 15:
            count = 0
            browser.close()
            browser = webdriver.Chrome()
        count += 1
        try:
            url = getUrl(browser, school, keywords)
            datatype = re.findall(r'image/.+?;', url)[0][6:-1]
            downloadImg(url, 'selected_data/{}/'.format(country), school, datatype)
        except:
            print('\nUniversity :' + school + '    failed\n')
        continue
        #print('University : ' + school + '     Success')

In [ ]:
main(keywords, 'USA', [x.strip() for x in df.Institution])

In [22]:
os.chdir('../../')

In [28]:
os.getcwd()

'/home/gs/git-repos/University-Logo-Deep-Learning-Community-Detection/selected_data/USA'

# 处理得到的数据
### 对爬取到的数据进行**切边**、**转换类型**、**Resize**等处理

In [3]:
from PIL import Image, ImageChops

def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

In [56]:
os.chdir('../')

In [57]:
os.listdir()

['USA',
 'Indian',
 'Korean',
 '.DS_Store',
 'France',
 'Netherlands',
 'Hong Kong',
 'Australia',
 'Germany',
 'Japan',
 'China',
 'UK',
 'Singapore',
 'New Zealand',
 'data_200',
 'Canada']

In [58]:
for sch in os.listdir():
    if (sch!='.DS_Store' and sch!='data_200'):
        os.chdir('./{0}/'.format(sch))
        country = os.path.basename(os.getcwd())
        
        for school in os.listdir():
        
            im = Image.open(school)
            result = trim(im)
            #if sch.endswith('png') or im.format == 'JPEG' or im.format == 'JPG':
            result = result.convert('RGB') # 转换格式为RGB
            result = result.resize((200, 200)) # resize
            os.remove(sch) # 删除原有图片

            result.save('../data_200/{}_{}.jpeg'.format(school[:-5], country) )
        os.chdir('../')